In [1]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("ANTHROPIC_API_KEY")
# _set_env("NEO4J_URI")
# _set_env("NEO4J_USERNAME")
# _set_env("NEO4J_PASSWORD")

In [2]:
from dotenv import load_dotenv  
from langchain.chains import GraphCypherQAChain  
from langchain_community.graphs import Neo4jGraph  
from langchain_core.documents import Document  
from langchain_experimental.graph_transformers import LLMGraphTransformer  
from langchain_anthropic import ChatAnthropic
  
load_dotenv()  
  
llm = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")  
  
llm_transformer = LLMGraphTransformer(llm=llm)

In [3]:
from langchain_community.chains.graph_qa.prompts import CYPHER_GENERATION_PROMPT

graph = Neo4jGraph()

def query_graph(graph, query):  
    chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True, allow_dangerous_requests = True, top_k = 5, cypher_prompt=CYPHER_GENERATION_PROMPT )  
    response = chain.invoke({"query": query})  
    return response

In [4]:
response = query_graph(graph, "What awards Marie Curie had?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.
{'query': 'What awards Marie Curie had?', 'result': "I'm afraid I don't have any information about what awards Marie Curie had. The provided information is empty, so I don't have enough details to give a helpful answer."}


In [5]:
text = """  
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.  
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.  
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.  
She was, in 1906, the first woman to become a professor at the University of Paris. She also won presidential awards.
"""

In [6]:
def add_knowledge_to_graph( text):
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(graph_documents)

    graph.refresh_schema()  # Refreshes the Neo4j graph schema information.
    
add_knowledge_to_graph(text)

In [7]:
response = query_graph(graph, "What awards Marie Curie had?")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: 'Marie Curie'})-[:WON_AWARD]->(a:Award)
RETURN a.id
Full Context:
[{'a.id': 'Nobel Prize'}, {'a.id': 'Presidential Awards'}]

> Finished chain.
{'query': 'What awards Marie Curie had?', 'result': 'According to the provided information, Marie Curie had the following awards:\n- Nobel Prize\n- Presidential Awards'}
